In [3]:
from bs4 import BeautifulSoup

import requests
import pandas as pd
from pprint import pprint as pp

In [4]:
def individual(listing_url):
    res = requests.get(listing_url).text
    soup = BeautifulSoup(res, 'html.parser')
    dict_of_all = {}

    ##Locations
    attr_box = soup.find('div', class_ = 'mapAndAttrs')
    bd_ba_sq = attr_box.find('p', class_ = 'attrgroup')
    misc_attr = bd_ba_sq.find_next('p', class_ = 'attrgroup')
    discription = soup.find('section', id = 'postingbody')
    
    ##Description Scrape
    description_dic = {'desc' : discription.text.split('\n')}
    description_dic = [i for i in description_dic['desc'] if i != '']
    description_dic = description_dic[1:]
    dict_of_all.update({'desc' : ', '.join(description_dic)})

    ##Attribute Scrape
    bed_bath = bd_ba_sq.find('span', class_ = 'shared-line-bubble')
    dict_of_all['bed_bath'] = bed_bath.text

    list_of_spans = misc_attr.find_all('span')
    attr_list = []
    for span in list_of_spans:
        text = span.text
        attr_list.append(text)
        attributes_str = ', '.join(attr_list)
        dict_of_all['attributes'] = attributes_str
    return dict_of_all

def page_collection(soup):

    ##Locations
    list_location = soup.find('ul', class_ = 'rows')
    soup_list_of_results = list_location.find_all('div', class_ = 'result-info')

    ##Description Scrape
    list_of_results = []
    for entry in soup_list_of_results:
        result_dict = {}
        title_a_tag = entry.find('a', class_ = 'result-title hdrlnk')
        meta_data = entry.find('span', class_ = 'result-meta')

        result_dict['link'] = title_a_tag['href']
        result_dict['title'] = title_a_tag.text
        result_dict['price'] = meta_data.find('span', class_ = 'result-price').text
        result_dict['city'] = meta_data.find_next('span', class_ = 'result-hood').text.replace('(', '').replace(')', '').strip().title()

        ##Squarefootage Parse
        housing_text = meta_data.find_next('span', class_ = 'housing').text
        dash_position = housing_text.index('-')
        square_footage_text = housing_text[dash_position:].replace('-', '').strip()
        result_dict['square_footage'] = square_footage_text
        
        list_of_results.append(result_dict)
    return list_of_results

In [17]:
def county_collector(location):
    start = '0'
    url = f'https://{location}.craigslist.org/search/apa?s={start}&availabilityMode=0&housing_type=6&sale_date=all+dates'
    html = requests.get(url).text
    soup = BeautifulSoup(html, 'html.parser')
    range_to = int(soup.find('span', class_ = 'rangeTo').text)
    total_count = int(soup.find('span', class_ = 'totalcount').text)
    all_entry_list = page_collection(soup)
    while total_count != range_to:
        try: 
            start = range_to
            url = f'https://{location}.craigslist.org/search/apa?s={start}&availabilityMode=0&housing_type=6&sale_date=all+dates'
            html = requests.get(url).text
            soup = BeautifulSoup(html, 'html.parser')
            page_list = page_collection(soup)
            all_entry_list.extend(page_list)
        except:
            print('end of list')
        range_to = int(soup.find('span', class_ = 'rangeTo').text)
    
    for post in all_entry_list:
        page_dict = individual(post['link'])
        post.update(page_dict)
    
    return all_entry_list


In [18]:
location = 'orangecounty'
all_list = county_collector(location)

print(len(all_list))
pp(all_list[0:5])

{'attributes': 'EV charging, dogs are OK - wooof, house, laundry on site, '
               'attached garage',
 'bed_bath': '3BR / 2Ba',
 'city': 'Costa Mesa',
 'desc': 'I’m looking to find a well employed family. to rent out my recently '
         'remodeled 3 bedroom with 2 bath house in Costa Mesa.  All new tiled '
         'bathrooms, new toilets, vanities and flooring. Nice washer and dryer '
         'in garage. The house, large 4 bedroom, 3 baths is located in the '
         'Mesa Verde Section of Costa Mesa, CA.  Close to beaches, bike paths, '
         'shopping centers, etc.  A very secure and quiet section of town. , '
         'The Master Bedroom size 15x15, has 1 attached bath, with new toilet, '
         'vanity and new tiles around shower.  The other 2 bedrooms are '
         'approx. 13x12.  The 2nd bath has all new tub, tiles, and toilet.  3 '
         'car parking in driveway, and 1 car in garage.  , The kitchen has '
         'been remodeled with new granite counterto

In [19]:
housing_rent = pd.DataFrame(all_list)
print(len(housing_rent))
housing_rent.head(n=10)

622


,link,title,price,city,square_footage,desc,bed_bath,attributes
0,https://orangecounty.craigslist.org/apa/d/cost...,"A quiet 3 Bd / 2 Ba House in Mesa Verde, remod...","$3,800",Costa Mesa,,I’m looking to find a well employed family. to...,3BR / 2Ba,"EV charging, dogs are OK - wooof, house, laund..."
1,https://orangecounty.craigslist.org/apa/d/newp...,"1 Bedroom Free Standing Cottage, One House off...","$2,700",Newport Beach,,Free Standing one bedroom cute beach house one...,1BR / 1Ba,"house, w/d in unit, no smoking, off-street par..."
2,https://orangecounty.craigslist.org/apa/d/lade...,MISSION VIEJO HOUSE,"$3,295",Mission Viejo,1200ft2,MISSION VIEJO HOME - 3 BEDROOMS - 2 BATHROOMS ...,3BR / 2Ba,"cats are OK - purrr, dogs are OK - wooof, hous..."
3,https://orangecounty.craigslist.org/apa/d/gard...,Garden Grove Family Home,"$3,250",Garden Grove,15000ft2,"Remodeled beautiful Garden Grove famil home, l...",3BR / 2Ba,"cats are OK - purrr, dogs are OK - wooof, floo..."
4,https://orangecounty.craigslist.org/apa/d/gard...,Beautiful house for rent,"$3,200",Westminster,2374ft2,Beautiful spacious 2 story home with office fu...,4BR / 2.5Ba,"\nthursday 2021-01-28\n, \nfriday 2021-01-29\n..."
5,https://orangecounty.craigslist.org/apa/d/irvi...,Adorable THREE ROOMTWO restroom bi-level TO HIRE,$782,Orange County,,Are you searching for the house that can shape...,4BR / 2Ba,"cats are OK - purrr, dogs are OK - wooof, hous..."
6,https://orangecounty.craigslist.org/apa/d/lake...,LAKE FOREST/IRVINE-MASTER SUITE-FURNISHED-REMO...,"$1,100",Lake Forest,1000ft2,This property is completely newly remodeled ma...,1BR / 1Ba,"furnished, house, laundry in bldg, off-street ..."
7,https://orangecounty.craigslist.org/apa/d/cost...,Charming 2b/2b house for rent in East Side CM,"$3,400",Costa Mesa,2000ft2,Charming and spacious 1930s bungalow for rent!...,2BR / 2Ba,"cats are OK - purrr, dogs are OK - wooof, floo..."
8,https://orangecounty.craigslist.org/apa/d/anah...,Beautiful Upgraded 1BR/1BA Guest House Include...,"$1,500",Anaheim,,Nice remodeled guest house/ in-law unit now av...,1BR / 1Ba,"house, laundry on site, no smoking, off-street..."
9,https://orangecounty.craigslist.org/apa/d/full...,VERY PRIVATE & SECURE DETACHED NEW GUEST HOUSE...,"$2,100",Fullerton,,"If you can see this ad, it is still available....",1BR / 1Ba,"flooring: tile, house, w/d in unit, no smoking..."


In [22]:
housing_rent.to_csv(r'data\{}.csv'.format(location), index=False)